In [2]:
from torchvision.models import *
import wandb
from sklearn.model_selection import train_test_split
import os,cv2
import numpy as np
import matplotlib.pyplot as plt
from torch.nn import *
import torch,torchvision
from tqdm import tqdm
device = 'cuda'
PROJECT_NAME = 'Vechicle-Dataset'

In [6]:
def load_data():
    labels = {}
    labels_r = {}
    data = []
    idx = -1
    for folder in tqdm(os.listdir('./data/')):
        idx += 1
        labels[folder] = idx
        labels_r[idx] = folder
        for file in os.listdir(f'./data/{folder}/'):
            img = cv2.imread(f'./data/{folder}/{file}')
            img = cv2.resize(img,(112,112))
            img = img / 255.0
            data.append([img,idx])
    X = []
    y = []
    for d in data:
        X.append(d[0])
        y.append(d[1])
    X_train,X_test,y_train,y_test = train_test_split(X,y,shuffle=False,test_size=0.0625)
    return X_train,X_test,y_train,y_test,X,y,labels,labels_r,idx,data

In [7]:
X_train,X_test,y_train,y_test,X,y,labels,labels_r,idx,data = load_data()

100%|███████████████████████████████████████████| 10/10 [00:00<00:00, 44.13it/s]


In [8]:
X_train = torch.from_numpy(np.array(X_train)).view(-1,3,112,112).to(device).float()
X_test = torch.from_numpy(np.array(X_test)).view(-1,3,112,112).to(device).float()
y_train = torch.from_numpy(np.array(y_train)).to(device).float()
y_test = torch.from_numpy(np.array(y_test)).to(device).float()

In [11]:
def get_loss(model,X,y,criterion):
    preds = model(X)
    loss = criterion(preds,y)
    return loss.item()

In [12]:
def get_accuracy(model,X,y):
    correct = 0
    total = 0
    preds = model(X)
    for pred,y_batch in zip(preds,y):
        pred = int(torch.argmax(pred))
        if pred == y_batch:
            correct += 1
        total += 1
    acc = round(correct/total,3)*100
    return acc

In [35]:
model = googlenet(pretrained=True).to(device)

In [36]:
model.fc = Linear(1024,idx+10)

In [37]:
idx

9

In [38]:
len(labels)

10

In [39]:
labels

{'train': 0,
 'bus': 1,
 'boat': 2,
 'cycle': 3,
 'truck': 4,
 'scooty': 5,
 'car': 6,
 'helicopter': 7,
 'plane': 8,
 'bike': 9}

In [40]:
criterion = CrossEntropyLoss()

In [41]:
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

In [42]:
epochs = 100

In [43]:
batch_size = 32

In [44]:
wandb.init(project=PROJECT_NAME,name='baseline-False')
for _ in tqdm(range(epochs)):
    for idx in range(0,len(X_train),batch_size):
        X_batch = X_train[idx:idx+batch_size].to(device)
        y_batch = y_train[idx:idx+batch_size].to(device)
        preds = model(X_batch)
        loss = criterion(preds,y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    wandb.log({'Loss':(get_loss(model,X_train,y_train,criterion)+get_loss(model,X_batch,y_batch,criterion))/2})
    wandb.log({'Val Loss':get_loss(model,X_test,y_test,criterion)})
    wandb.log({'Acc':(get_accuracy(model,X_train,y_train)+get_accuracy(model,X_batch,y_batch))/2})
    wandb.log({'Val ACC':get_accuracy(model,X_test,y_test)})
wandb.finish()

wandb: Currently logged in as: ranuga-d (use `wandb login --relogin` to force relogin)


  0%|                                                   | 0/100 [00:00<?, ?it/s]


RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same